In [9]:
import pandas as pd
from collections import Counter
from pprint import pprint
import math
import numpy as np
df = pd.read_csv('3.csv')

In [10]:
target ='play'
attribute_names = ['outlook', 'temp', 'humidity', 'wind']

In [11]:
def entropy(probs):  
    return sum( [-prob*math.log(prob, 2) for prob in probs])
def entropy_of_list(ls,value):  
    cnt = Counter(x for x in ls)
    total_instances = len(ls)  
    probs = [x / total_instances for x in cnt.values()] 
    return entropy(probs)

In [12]:
def information_gain(df, split_attribute, target_attribute,battr):
    df_split = df.groupby(split_attribute) 
    glist=[]
    for gname,group in df_split:
        glist.append(gname) 
    glist.reverse()
    nobs = len(df.index) * 1.0   
    df_agg1=df_split.agg({target_attribute:lambda x:entropy_of_list(x, glist.pop())})
    df_agg2=df_split.agg({target_attribute :lambda x:len(x)/nobs})
    df_agg1.columns=['Entropy']
    df_agg2.columns=['Proportion']
    new_entropy = sum( df_agg1['Entropy'] * df_agg2['Proportion'])
    if battr !='S':
        old_entropy = entropy_of_list(df[target_attribute],'S-'+df.iloc[0][df.columns.get_loc(battr)])
    else:
        old_entropy = entropy_of_list(df[target_attribute],battr)
    return old_entropy - new_entropy

In [13]:
def id3(df, target_attribute, attribute_names, default_class=None,default_attr='S'):
    cnt = Counter(x for x in df[target_attribute])
    if len(cnt) == 1:
        return next(iter(cnt))
    elif df.empty or (not attribute_names):
        return default_class
    else:
        default_class = max(cnt.keys())
        gainz=[]
        for attr in attribute_names:
            ig= information_gain(df, attr, target_attribute,default_attr)
            gainz.append(ig)
        index_of_max = gainz.index(max(gainz))               
        best_attr = attribute_names[index_of_max]            
        tree = {best_attr:{}}
        remaining_attribute_names =[i for i in attribute_names if i != best_attr]
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,target_attribute, remaining_attribute_names,default_class,best_attr)
            tree[best_attr][attr_val] = subtree
        return tree

In [14]:
tree = id3(df,target,attribute_names)
print("\nThe Resultant Decision Tree is:")
pprint(tree)


The Resultant Decision Tree is:
{'outlook': {'overcast': 'yes',
             'rain': {'wind': {'strong': 'no', 'weak': 'yes'}},
             'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}
